# Experiment 002: LightGBM with Per-Target Optimization

Since TabPFN requires authentication, pivoting to LightGBM which was mentioned in the strategy:
- Per-target LightGBM regressors (3 separate models)
- Early stopping with patience
- Same Arrhenius kinetics features + Spange descriptors
- TTA for mixed solvent symmetry

In [1]:
import numpy as np
import pandas as pd
import torch
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"LightGBM version: {lgb.__version__}")

LightGBM version: 4.5.0


In [2]:
# Data loading utilities
DATA_PATH = '/home/data'

INPUT_LABELS_FULL_SOLVENT = [
    "Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%",
]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent_name in sorted(X["SOLVENT NAME"].unique()):
        train_mask = X["SOLVENT NAME"] != solvent_name
        yield (X[train_mask], Y[train_mask]), (X[~train_mask], Y[~train_mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates().sort_values(["SOLVENT A NAME", "SOLVENT B NAME"])
    for _, row in ramps.iterrows():
        train_mask = (X[["SOLVENT A NAME", "SOLVENT B NAME"]] != row).any(axis=1)
        yield (X[train_mask], Y[train_mask]), (X[~train_mask], Y[~train_mask])

# Load Spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f"Spange descriptors shape: {SPANGE_DF.shape}")

Spange descriptors shape: (26, 13)


In [3]:
# Featurizer with Arrhenius kinetics
class KineticFeaturizer:
    """Featurizer with Arrhenius kinetics features."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        self.feats_dim = self.featurizer.shape[1] + 2 + 3  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        # Solvent features
        if self.mixed:
            A = self.featurizer.loc[X["SOLVENT A NAME"]].values
            B = self.featurizer.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if flip:
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = self.featurizer.loc[X["SOLVENT NAME"]].values
            
        return np.hstack([X_kinetic, X_chem])

In [4]:
# LightGBM Model with TTA support
class LightGBMModel:
    def __init__(self, data='single', use_tta=True):
        self.data_type = data
        self.use_tta = use_tta and (data == 'full')
        self.featurizer = KineticFeaturizer(mixed=(data=='full'))
        self.models = []
        
        # LightGBM parameters
        self.params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': 0.03,
            'num_leaves': 31,
            'max_depth': 6,
            'min_child_samples': 10,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1,
            'seed': 42,
        }
        
    def train_model(self, X_train, y_train):
        X_feats = self.featurizer.featurize(X_train, flip=False)
        
        if self.data_type == 'full' and self.use_tta:
            # Data augmentation with symmetric solvent swapping
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_feats, X_flip])
            y_all = np.vstack([y_train.values, y_train.values])
        else:
            X_all = X_feats
            y_all = y_train.values
        
        self.models = []
        for i in range(3):  # 3 targets
            # Create train/val split for early stopping
            X_tr, X_val, y_tr, y_val = train_test_split(
                X_all, y_all[:, i], test_size=0.15, random_state=42
            )
            
            train_data = lgb.Dataset(X_tr, label=y_tr)
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
            model = lgb.train(
                self.params,
                train_data,
                num_boost_round=1000,
                valid_sets=[val_data],
                callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
            )
            self.models.append(model)
    
    def predict(self, X_test):
        X_feats = self.featurizer.featurize(X_test, flip=False)
        
        if self.use_tta:
            X_flip = self.featurizer.featurize(X_test, flip=True)
            preds_std = np.column_stack([m.predict(X_feats) for m in self.models])
            preds_flip = np.column_stack([m.predict(X_flip) for m in self.models])
            preds = (preds_std + preds_flip) / 2
        else:
            preds = np.column_stack([m.predict(X_feats) for m in self.models])
        
        preds = np.clip(preds, 0, 1)
        return torch.tensor(preds)

In [5]:
# Test on single fold
print("Testing LightGBM on single fold...")
X, Y = load_data("single_solvent")
print(f"Single solvent data: {X.shape[0]} samples")

split_gen = generate_leave_one_out_splits(X, Y)
(train_X, train_Y), (test_X, test_Y) = next(split_gen)
print(f"Train: {len(train_X)}, Test: {len(test_X)}")

model = LightGBMModel(data='single')
model.train_model(train_X, train_Y)

preds = model.predict(test_X)
print(f"Predictions shape: {preds.shape}")
print(f"Predictions range: [{preds.min():.4f}, {preds.max():.4f}]")

rmse = np.sqrt(((preds.numpy() - test_Y.values) ** 2).mean())
print(f"Single fold RMSE: {rmse:.4f}")

Testing LightGBM on single fold...
Single solvent data: 656 samples
Train: 619, Test: 37


Predictions shape: torch.Size([37, 3])
Predictions range: [0.0013, 0.9125]
Single fold RMSE: 0.2166


In [6]:
# Full CV for single solvent task
print("\n" + "="*50)
print("TASK 0: Single Solvent (Leave-One-Solvent-Out CV)")
print("="*50)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
fold_rmses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LightGBMModel(data='single')
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    
    fold_rmse = np.sqrt(((predictions - test_Y.values) ** 2).mean())
    fold_rmses.append(fold_rmse)
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"\nSingle Solvent CV RMSE: {np.mean(fold_rmses):.5f} ± {np.std(fold_rmses):.5f}")


TASK 0: Single Solvent (Leave-One-Solvent-Out CV)


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:00<00:15,  1.48it/s]

  8%|▊         | 2/24 [00:01<00:12,  1.73it/s]

 12%|█▎        | 3/24 [00:02<00:15,  1.33it/s]

...

 92%|█████████▏| 22/24 [00:16<00:01,  1.36it/s]

 96%|█████████▌| 23/24 [00:17<00:00,  1.44it/s]

100%|██████████| 24/24 [00:17<00:00,  1.43it/s]

100%|██████████| 24/24 [00:17<00:00,  1.35it/s]


Single Solvent CV RMSE: 0.09971 ± 0.04900


In [7]:
# Full CV for full data task
print("\n" + "="*50)
print("TASK 1: Full Data (Leave-One-Ramp-Out CV)")
print("="*50)

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []
fold_rmses_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LightGBMModel(data='full', use_tta=True)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    
    fold_rmse = np.sqrt(((predictions - test_Y.values) ** 2).mean())
    fold_rmses_full.append(fold_rmse)
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

submission_full_data = pd.DataFrame(all_predictions)
print(f"\nFull Data CV RMSE: {np.mean(fold_rmses_full):.5f} ± {np.std(fold_rmses_full):.5f}")


TASK 1: Full Data (Leave-One-Ramp-Out CV)


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:01<00:21,  1.75s/it]

 15%|█▌        | 2/13 [00:03<00:20,  1.88s/it]

 23%|██▎       | 3/13 [00:05<00:17,  1.75s/it]

...

 85%|████████▍ | 11/13 [00:20<00:03,  1.95s/it]

 92%|█████████▏| 12/13 [00:22<00:01,  1.91s/it]

100%|██████████| 13/13 [00:24<00:00,  1.96s/it]

100%|██████████| 13/13 [00:24<00:00,  1.88s/it]


Full Data CV RMSE: 0.10067 ± 0.03152


In [8]:
# Save submission
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"

submission.to_csv("/home/code/experiments/002_lightgbm/submission.csv", index=True)
submission.to_csv("/home/submission/submission.csv", index=True)

print(f"\nSubmission saved with {len(submission)} rows")


Submission saved with 1883 rows


In [9]:
# Final summary
print("\n" + "="*50)
print("FINAL RESULTS - LightGBM")
print("="*50)
print(f"Single Solvent CV RMSE: {np.mean(fold_rmses):.5f} ± {np.std(fold_rmses):.5f}")
print(f"Full Data CV RMSE: {np.mean(fold_rmses_full):.5f} ± {np.std(fold_rmses_full):.5f}")

overall_rmse = (np.mean(fold_rmses) + np.mean(fold_rmses_full)) / 2
print(f"\nOverall CV RMSE: {overall_rmse:.5f}")

print(f"\nComparison:")
print(f"  Baseline MLP: 0.08819")
print(f"  LightGBM:     {overall_rmse:.5f}")
print(f"  Target:       0.04740")


FINAL RESULTS - LightGBM
Single Solvent CV RMSE: 0.09971 ± 0.04900
Full Data CV RMSE: 0.10067 ± 0.03152

Overall CV RMSE: 0.10019

Comparison:
  Baseline MLP: 0.08819
  LightGBM:     0.10019
  Target:       0.04740
